In [ ]:
import os
import random
import cv2 as cv
import numpy as np
import tensorflow as tf
from sklearn import metrics
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten

In [ ]:
# Upload train/test data

np.random.seed(7)
female = []
male = []

for i in os.listdir('C:/Users/72684/Desktop/Reinforcement Learning/Female'):
    path = 'C:/Users/72684/Desktop/Reinforcement Learning/Female/{}'.format(i)
    female.append([cv.imread(path, cv.IMREAD_COLOR) / 255, 0])

for i in os.listdir('C:/Users/72684/Desktop/Reinforcement Learning/Male'):
    path = 'C:/Users/72684/Desktop/Reinforcement Learning/Male/{}'.format(i)
    male.append([cv.imread(path, cv.IMREAD_COLOR) / 255, 1])

female = np.array(female)
male = np.array(male)

np.random.seed(7)
f_trn, f_tst = train_test_split(female, test_size = 0.2)
np.random.seed(7)
m_trn, m_tst = train_test_split(male, test_size = 0.2)

train = []
test = []

for i in range(len(f_trn)):
    train.append(f_trn[i])
    
for i in range(len(m_trn)):
    train.append(m_trn[i])

for i in range(len(f_tst)):
    test.append(f_tst[i])

for i in range(len(m_tst)):
    test.append(m_tst[i])

random.seed(7)
random.shuffle(train)
random.seed(7)
random.shuffle(test)

train = np.array(train)
test = np.array(test)

trn_d = []
tst_d = []

trn_l = np.zeros(len(train))
tst_l = np.zeros(len(test))

for i in range(len(train)):
    trn_l[i] = train[i, 1]
    trn_d.append(train[i, 0])

for i in range(len(test)):
    tst_l[i] = test[i, 1]
    tst_d.append(test[i, 0])

trn_d = np.array(trn_d)
tst_d = np.array(tst_d)

In [ ]:
# Define your model and its structural hyperparameters there

def build_model(krnl1, krnl2, krnl3, dns_lr, maps):
    tf.random.set_seed(7)
    
    mdl = Sequential()
    
    mdl.add(Conv2D(maps, krnl1, activation = 'relu', input_shape = (48, 48, 3), padding = "same"))
    mdl.add(MaxPooling2D(2, 2))
    
    mdl.add(Conv2D(2 * maps, krnl2, activation = 'relu', padding = "same"))
    mdl.add(MaxPooling2D(2, 2))
    
    mdl.add(Conv2D(4 * maps, krnl3, activation = 'relu', padding = "same"))
    
    mdl.add(Flatten())
    mdl.add(Dense(dns_lr, activation = 'relu'))
    mdl.add(Dense(1, activation = 'sigmoid'))
    
    mdl.compile(loss = 'binary_crossentropy', optimizer = 'adagrad', metrics = ['accuracy'])
    
    return mdl

## Greedy

In [ ]:
# Hyperparameters search algorithm

def hyp_prm_fit(epochs, btch_sz, krnl1, krnl2, krnl3, dns_lr, maps, trn_d, trn_l, tst_d, tst_l): # Define your system's
    h_p = np.array([epochs, btch_sz, dns_lr, maps, krnl1, krnl2, krnl3])                         # hyperparameters including
    steps_cntns = np.array([0.6, 0.8, 1, 1.2, 1.4])                                              # train hyperparameters
    steps_dscrt = np.array([3, 5, 7, 9], int) # steps_cntns steps for continuous values (not limited with some set of values)
    dt_tp = np.array(['int', 'int', 'int', 'int']) # steps_dscrt for discrete values (possible kernel sizes and so on)
    acc = 0 # dt_tp is to set continuous value's datatype for each hyperparameter
    cnt = 0
    acc_hst = [] # Change names and amount of variables below if you have another set of hyperparameters
    epchs_hst = []
    btch_sz_hst = []
    krnl1_hst = []
    krnl2_hst = []
    krnl3_hst = []
    dns_lr_hst = []
    maps_hst = []
    
    mdl = build_model((krnl1, krnl1), (krnl2, krnl2), (krnl3, krnl3), dns_lr, maps) # Set structural hyperparameters
    tf.random.set_seed(7) # Set train hyperparameters below
    mdl.fit(trn_d, trn_l, epochs = epochs, batch_size = btch_sz, validation_data = (tst_d, tst_l), verbose = False)
    acc = mdl.evaluate(tst_d, tst_l, verbose = False)[1]
    acc_hst.append(acc) # Change names and amount of variables below if you have another set of hyperparameters
    epchs_hst.append(epochs)
    btch_sz_hst.append(btch_sz)
    krnl1_hst.append(krnl1)
    krnl2_hst.append(krnl2)
    krnl3_hst.append(krnl3)
    dns_lr_hst.append(dns_lr)
    maps_hst.append(maps)
    print(acc)
    
    for episode in range(10): # Amount of episodes to find hyperparameters
        for i in range(len(h_p)):
            if i <= 3: # This value + 1 corresponds to amount of continuous hyperparameters
                h_p_fld = np.zeros(len(steps_cntns))
                h_p_fld = np.array(list(set(np.array(h_p[i] * steps_cntns, dt_tp[i]))))
                new_acc = np.zeros(len(h_p_fld))
                rsrv = h_p[i]
                print(h_p)
                print(h_p_fld)
            
            else: # There the work with discrete hyperparameters starts
                h_p_fld = np.zeros(len(steps_dscrt))
                h_p_fld = np.array(steps_dscrt)
                new_acc = np.zeros(len(h_p_fld))
                rsrv = h_p[i]
                print(h_p)
                print(h_p_fld)
            
            for val in range(len(h_p_fld)):
                h_p[i] = h_p_fld[val] # h_p stores current values of hyperparameters for training
                
                mdl = build_model((h_p[4], h_p[4]), (h_p[5], h_p[5]), (h_p[6], h_p[6]), h_p[2], h_p[3])
                tf.random.set_seed(7)
                mdl.fit(trn_d, trn_l, epochs = h_p[0], batch_size = h_p[1], validation_data = (tst_d, tst_l), verbose = False)
                new_acc[val] = mdl.evaluate(tst_d, tst_l, verbose = False)[1]
            
            
            if np.max(new_acc) > acc:
                cnt = 0
            
            else:
                cnt += 1
            
            if h_p_fld[np.argmax(new_acc)] == rsrv:
                res_acc = np.copy(new_acc)
                res_acc[np.argmax(new_acc)] = 0
                
                if np.max(res_acc) == np.max(new_acc):
                    h_p[i] = h_p_fld[np.argmax(res_acc)]
                    acc_hst.append(np.max(res_acc))
                    acc = np.max(res_acc)
                
                else:
                    h_p[i] = h_p_fld[np.argmax(new_acc)]
                    acc_hst.append(np.max(new_acc))
                    acc = np.max(new_acc)
            
            else:
                h_p[i] = h_p_fld[np.argmax(new_acc)]
                acc_hst.append(np.max(new_acc))
                acc = np.max(new_acc)
            
            print(new_acc)
            
            if i == 0: # Change names and amount of variables below if you have another set of hyperparameters
                epchs_hst.append(h_p[i])
            
            if i == 1:
                btch_sz_hst.append(h_p[i])
            
            if i == 2:
                dns_lr_hst.append(h_p[i])
            
            if i == 3:
                maps_hst.append(h_p[i])
            
            if i == 4:
                krnl1_hst.append(h_p[i])
            
            if i == 5:
                krnl2_hst.append(h_p[i])
            
            if i == 6:
                krnl3_hst.append(h_p[i])
            
            if cnt > 2 * len(h_p): # Search breaking criterion
                break
        
        if cnt > 2 * len(h_p):
            break
    
    acc_hst = np.array(acc_hst) # Change names and amount of variables below if you have another set of hyperparameters
    epchs_hst = np.array(epchs_hst)
    btch_sz_hst = np.array(btch_sz_hst)
    krnl1_hst = np.array(krnl1_hst)
    krnl2_hst = np.array(krnl2_hst)
    krnl3_hst = np.array(krnl3_hst)
    dns_lr_hst = np.array(dns_lr_hst)
    maps_hst = np.array(maps_hst)
    print(h_p)
    
    plt.plot(range(len(acc_hst)), acc_hst) # Change amount of plots below with respect to amount of hyperparameters (except acc)
    plt.grid()
    plt.xlabel('Hyperparameter changes')
    plt.ylabel('Accuracy')
    plt.show()
    
    plt.plot(range(len(epchs_hst)), epchs_hst)
    plt.grid()
    plt.xlabel('Hyperparameter epochs changes')
    plt.ylabel('Amount of epochs')
    plt.show()
    
    plt.plot(range(len(btch_sz_hst)), btch_sz_hst)
    plt.grid()
    plt.xlabel('Hyperparameter batch size changes')
    plt.ylabel('Batch size')
    plt.show()
    
    plt.plot(range(len(krnl1_hst)), krnl1_hst)
    plt.grid()
    plt.xlabel('Hyperparameter kernel 1 size changes')
    plt.ylabel('Kernel 1 size')
    plt.show()
    
    plt.plot(range(len(krnl2_hst)), krnl2_hst)
    plt.grid()
    plt.xlabel('Hyperparameter kernel 2 size changes')
    plt.ylabel('Kernel 2 size')
    plt.show()
    
    plt.plot(range(len(krnl3_hst)), krnl3_hst)
    plt.grid()
    plt.xlabel('Hyperparameter kernel 3 size changes')
    plt.ylabel('Kernel 3 size')
    plt.show()
    
    plt.plot(range(len(dns_lr_hst)), dns_lr_hst)
    plt.grid()
    plt.xlabel('Hyperparameter size of dense layer changes')
    plt.ylabel('Size of dense layer')
    plt.show()
    
    plt.plot(range(len(maps_hst)), maps_hst)
    plt.grid()
    plt.xlabel('Hyperparameter basic amount of maps changes')
    plt.ylabel('Basic amount of maps')
    plt.show()
    
    return np.array([acc_hst, epchs_hst, btch_sz_hst, krnl1_hst, krnl2_hst, krnl3_hst, dns_lr_hst, maps_hst, h_p])

In [ ]:
res_1 = hyp_prm_fit(15, 15, 3, 3, 3, 100, 16, trn_d, trn_l, tst_d, tst_l) # Set initial value of each hyperparameter

## e-greedy

In [ ]:
def hyp_prm_fit(epochs, btch_sz, krnl1, krnl2, krnl3, dns_lr, maps, trn_d, trn_l, tst_d, tst_l):
    h_p = np.array([epochs, btch_sz, dns_lr, maps, krnl1, krnl2, krnl3])
    steps_cntns = np.array([0.6, 0.8, 1, 1.2, 1.4])
    steps_dscrt = np.array([3, 5, 7, 9], int)
    dt_tp = np.array(['int', 'int', 'int', 'int'])
    acc = 0
    cnt = 0
    acc_hst = []
    epchs_hst = []
    btch_sz_hst = []
    krnl1_hst = []
    krnl2_hst = []
    krnl3_hst = []
    dns_lr_hst = []
    maps_hst = []
    
    mdl = build_model((krnl1, krnl1), (krnl2, krnl2), (krnl3, krnl3), dns_lr, maps)
    tf.random.set_seed(7)
    mdl.fit(trn_d, trn_l, epochs = epochs, batch_size = btch_sz, validation_data = (tst_d, tst_l), verbose = False)
    acc = mdl.evaluate(tst_d, tst_l, verbose = False)[1]
    acc_hst.append(acc)
    epchs_hst.append(epochs)
    btch_sz_hst.append(btch_sz)
    krnl1_hst.append(krnl1)
    krnl2_hst.append(krnl2)
    krnl3_hst.append(krnl3)
    dns_lr_hst.append(dns_lr)
    maps_hst.append(maps)
    print(acc)
    
    for episode in range(10):
        for i in range(len(h_p)):
            if i <= 3:
                h_p_fld = np.zeros(len(steps_cntns))
                h_p_fld = np.array(list(set(np.array(h_p[i] * steps_cntns, dt_tp[i]))))
                new_acc = np.zeros(len(h_p_fld))
                rsrv = h_p[i]
                print(h_p)
                print(h_p_fld)
            
            else:
                h_p_fld = np.zeros(len(steps_dscrt))
                h_p_fld = np.array(steps_dscrt)
                new_acc = np.zeros(len(h_p_fld))
                rsrv = h_p[i]
                print(h_p)
                print(h_p_fld)
            
            for val in range(len(h_p_fld)):
                h_p[i] = h_p_fld[val]
                
                mdl = build_model((h_p[4], h_p[4]), (h_p[5], h_p[5]), (h_p[6], h_p[6]), h_p[2], h_p[3])
                tf.random.set_seed(7)
                mdl.fit(trn_d, trn_l, epochs = h_p[0], batch_size = h_p[1], validation_data = (tst_d, tst_l), verbose = False)
                new_acc[val] = mdl.evaluate(tst_d, tst_l, verbose = False)[1]
            
            
            if np.random.rand() > 0.95:
                chc = np.random.choice(len(h_p_fld))
                
                if new_acc[chc] > acc:
                    cnt = 0
                
                else:
                    cnt += 1
                
                h_p[i] = h_p_fld[chc]
                acc_hst.append(new_acc[chc])
                acc = new_acc[chc]
            
            else:
                if np.max(new_acc) > acc:
                    cnt = 0
                
                else:
                    cnt += 1
                
                if h_p_fld[np.argmax(new_acc)] == rsrv:
                    res_acc = np.copy(new_acc)
                    res_acc[np.argmax(new_acc)] = 0
                    
                    if np.max(res_acc) == np.max(new_acc):
                        h_p[i] = h_p_fld[np.argmax(res_acc)]
                        acc_hst.append(np.max(res_acc))
                        acc = np.max(res_acc)
                    
                    else:
                        h_p[i] = h_p_fld[np.argmax(new_acc)]
                        acc_hst.append(np.max(new_acc))
                        acc = np.max(new_acc)
                
                else:
                    h_p[i] = h_p_fld[np.argmax(new_acc)]
                    acc_hst.append(np.max(new_acc))
                    acc = np.max(new_acc)
            
            print(new_acc)
            
            if i == 0:
                epchs_hst.append(h_p[i])
            
            if i == 1:
                btch_sz_hst.append(h_p[i])
            
            if i == 2:
                dns_lr_hst.append(h_p[i])
            
            if i == 3:
                maps_hst.append(h_p[i])
            
            if i == 4:
                krnl1_hst.append(h_p[i])
            
            if i == 5:
                krnl2_hst.append(h_p[i])
            
            if i == 6:
                krnl3_hst.append(h_p[i])
            
            if cnt > 2 * len(h_p):
                break
        
        if cnt > 2 * len(h_p):
            break
    
    acc_hst = np.array(acc_hst)
    epchs_hst = np.array(epchs_hst)
    btch_sz_hst = np.array(btch_sz_hst)
    krnl1_hst = np.array(krnl1_hst)
    krnl2_hst = np.array(krnl2_hst)
    krnl3_hst = np.array(krnl3_hst)
    dns_lr_hst = np.array(dns_lr_hst)
    maps_hst = np.array(maps_hst)
    print(h_p)
    
    plt.plot(range(len(acc_hst)), acc_hst)
    plt.grid()
    plt.xlabel('Hyperparameter changes')
    plt.ylabel('Accuracy')
    plt.show()
    
    plt.plot(range(len(epchs_hst)), epchs_hst)
    plt.grid()
    plt.xlabel('Hyperparameter epochs changes')
    plt.ylabel('Amount of epochs')
    plt.show()
    
    plt.plot(range(len(btch_sz_hst)), btch_sz_hst)
    plt.grid()
    plt.xlabel('Hyperparameter batch size changes')
    plt.ylabel('Batch size')
    plt.show()
    
    plt.plot(range(len(krnl1_hst)), krnl1_hst)
    plt.grid()
    plt.xlabel('Hyperparameter kernel 1 size changes')
    plt.ylabel('Kernel 1 size')
    plt.show()
    
    plt.plot(range(len(krnl2_hst)), krnl2_hst)
    plt.grid()
    plt.xlabel('Hyperparameter kernel 2 size changes')
    plt.ylabel('Kernel 2 size')
    plt.show()
    
    plt.plot(range(len(krnl3_hst)), krnl3_hst)
    plt.grid()
    plt.xlabel('Hyperparameter kernel 3 size changes')
    plt.ylabel('Kernel 3 size')
    plt.show()
    
    plt.plot(range(len(dns_lr_hst)), dns_lr_hst)
    plt.grid()
    plt.xlabel('Hyperparameter size of dense layer changes')
    plt.ylabel('Size of dense layer')
    plt.show()
    
    plt.plot(range(len(maps_hst)), maps_hst)
    plt.grid()
    plt.xlabel('Hyperparameter basic amount of maps changes')
    plt.ylabel('Basic amount of maps')
    plt.show()
    
    return np.array([acc_hst, epchs_hst, btch_sz_hst, krnl1_hst, krnl2_hst, krnl3_hst, dns_lr_hst, maps_hst, h_p])

In [ ]:
res_2 = hyp_prm_fit(15, 15, 3, 3, 3, 100, 16, trn_d, trn_l, tst_d, tst_l)

## Decaying e-greedy

In [ ]:
def hyp_prm_fit(epochs, btch_sz, krnl1, krnl2, krnl3, dns_lr, maps, trn_d, trn_l, tst_d, tst_l):
    h_p = np.array([epochs, btch_sz, dns_lr, maps, krnl1, krnl2, krnl3])
    steps_cntns = np.array([0.6, 0.8, 1, 1.2, 1.4])
    steps_dscrt = np.array([3, 5, 7, 9], int)
    dt_tp = np.array(['int', 'int', 'int', 'int'])
    acc = 0
    cnt = 0
    acc_hst = []
    epchs_hst = []
    btch_sz_hst = []
    krnl1_hst = []
    krnl2_hst = []
    krnl3_hst = []
    dns_lr_hst = []
    maps_hst = []
    
    mdl = build_model((krnl1, krnl1), (krnl2, krnl2), (krnl3, krnl3), dns_lr, maps)
    tf.random.set_seed(7)
    mdl.fit(trn_d, trn_l, epochs = epochs, batch_size = btch_sz, validation_data = (tst_d, tst_l), verbose = False)
    acc = mdl.evaluate(tst_d, tst_l, verbose = False)[1]
    acc_hst.append(acc)
    epchs_hst.append(epochs)
    btch_sz_hst.append(btch_sz)
    krnl1_hst.append(krnl1)
    krnl2_hst.append(krnl2)
    krnl3_hst.append(krnl3)
    dns_lr_hst.append(dns_lr)
    maps_hst.append(maps)
    print(acc)
    
    for episode in range(10):
        for i in range(len(h_p)):
            if i <= 3:
                h_p_fld = np.zeros(len(steps_cntns))
                h_p_fld = np.array(list(set(np.array(h_p[i] * steps_cntns, dt_tp[i]))))
                new_acc = np.zeros(len(h_p_fld))
                rsrv = h_p[i]
                print(h_p)
                print(h_p_fld)
            
            else:
                h_p_fld = np.zeros(len(steps_dscrt))
                h_p_fld = np.array(steps_dscrt)
                new_acc = np.zeros(len(h_p_fld))
                rsrv = h_p[i]
                print(h_p)
                print(h_p_fld)
            
            for val in range(len(h_p_fld)):
                h_p[i] = h_p_fld[val]
                
                mdl = build_model((h_p[4], h_p[4]), (h_p[5], h_p[5]), (h_p[6], h_p[6]), h_p[2], h_p[3])
                tf.random.set_seed(7)
                mdl.fit(trn_d, trn_l, epochs = h_p[0], batch_size = h_p[1], validation_data = (tst_d, tst_l), verbose = False)
                new_acc[val] = mdl.evaluate(tst_d, tst_l, verbose = False)[1]
            
            
            if np.random.rand() > (1 - 0.2 / (episode + 1)):
                chc = np.random.choice(len(h_p_fld))
                
                if new_acc[chc] > acc:
                    cnt = 0
                
                else:
                    cnt += 1
                
                h_p[i] = h_p_fld[chc]
                acc_hst.append(new_acc[chc])
                acc = new_acc[chc]
            
            else:
                if np.max(new_acc) > acc:
                    cnt = 0
                
                else:
                    cnt += 1
                
                if h_p_fld[np.argmax(new_acc)] == rsrv:
                    res_acc = np.copy(new_acc)
                    res_acc[np.argmax(new_acc)] = 0
                    
                    if np.max(res_acc) == np.max(new_acc):
                        h_p[i] = h_p_fld[np.argmax(res_acc)]
                        acc_hst.append(np.max(res_acc))
                        acc = np.max(res_acc)
                    
                    else:
                        h_p[i] = h_p_fld[np.argmax(new_acc)]
                        acc_hst.append(np.max(new_acc))
                        acc = np.max(new_acc)
                
                else:
                    h_p[i] = h_p_fld[np.argmax(new_acc)]
                    acc_hst.append(np.max(new_acc))
                    acc = np.max(new_acc)
            
            print(new_acc)
            
            if i == 0:
                epchs_hst.append(h_p[i])
            
            if i == 1:
                btch_sz_hst.append(h_p[i])
            
            if i == 2:
                dns_lr_hst.append(h_p[i])
            
            if i == 3:
                maps_hst.append(h_p[i])
            
            if i == 4:
                krnl1_hst.append(h_p[i])
            
            if i == 5:
                krnl2_hst.append(h_p[i])
            
            if i == 6:
                krnl3_hst.append(h_p[i])
            
            if cnt > 2 * len(h_p):
                break
        
        if cnt > 2 * len(h_p):
            break
    
    acc_hst = np.array(acc_hst)
    epchs_hst = np.array(epchs_hst)
    btch_sz_hst = np.array(btch_sz_hst)
    krnl1_hst = np.array(krnl1_hst)
    krnl2_hst = np.array(krnl2_hst)
    krnl3_hst = np.array(krnl3_hst)
    dns_lr_hst = np.array(dns_lr_hst)
    maps_hst = np.array(maps_hst)
    print(h_p)
    
    plt.plot(range(len(acc_hst)), acc_hst)
    plt.grid()
    plt.xlabel('Hyperparameter changes')
    plt.ylabel('Accuracy')
    plt.show()
    
    plt.plot(range(len(epchs_hst)), epchs_hst)
    plt.grid()
    plt.xlabel('Hyperparameter epochs changes')
    plt.ylabel('Amount of epochs')
    plt.show()
    
    plt.plot(range(len(btch_sz_hst)), btch_sz_hst)
    plt.grid()
    plt.xlabel('Hyperparameter batch size changes')
    plt.ylabel('Batch size')
    plt.show()
    
    plt.plot(range(len(krnl1_hst)), krnl1_hst)
    plt.grid()
    plt.xlabel('Hyperparameter kernel 1 size changes')
    plt.ylabel('Kernel 1 size')
    plt.show()
    
    plt.plot(range(len(krnl2_hst)), krnl2_hst)
    plt.grid()
    plt.xlabel('Hyperparameter kernel 2 size changes')
    plt.ylabel('Kernel 2 size')
    plt.show()
    
    plt.plot(range(len(krnl3_hst)), krnl3_hst)
    plt.grid()
    plt.xlabel('Hyperparameter kernel 3 size changes')
    plt.ylabel('Kernel 3 size')
    plt.show()
    
    plt.plot(range(len(dns_lr_hst)), dns_lr_hst)
    plt.grid()
    plt.xlabel('Hyperparameter size of dense layer changes')
    plt.ylabel('Size of dense layer')
    plt.show()
    
    plt.plot(range(len(maps_hst)), maps_hst)
    plt.grid()
    plt.xlabel('Hyperparameter basic amount of maps changes')
    plt.ylabel('Basic amount of maps')
    plt.show()
    
    return np.array([acc_hst, epchs_hst, btch_sz_hst, krnl1_hst, krnl2_hst, krnl3_hst, dns_lr_hst, maps_hst, h_p])

In [ ]:
res_3 = hyp_prm_fit(15, 15, 3, 3, 3, 100, 16, trn_d, trn_l, tst_d, tst_l)

In [ ]:
plt.plot(range(len(res_1[0])), res_1[0], linewidth = 4)
plt.plot(range(len(res_2[0])), res_2[0], linewidth = 4)
plt.plot(range(len(res_3[0])), res_3[0], linewidth = 4)
plt.grid()
plt.xlabel('Amount of model estimations')
plt.ylabel('Accuracy of the model')
plt.legend(['Greedy', 'e-greedy', 'Decaying e-greedy'])
plt.show()

In [ ]:
print('Greedy: amount of epochs - ' + str(res_1[8][0]) + ', batch size - ' + str(res_1[8][1]) + ', dense layer size - ' + \
    str(res_1[8][2]) + ', amount of maps - ' + str(res_1[8][3]) + ', kernel 1 size - ' + str(res_1[8][4]) + \
    ', kernel 2 size - ' + str(res_1[8][5]) + ', kernel 3 size - ' + str(res_1[8][6]) + '\n')
print('e-greedy: amount of epochs - ' + str(res_2[8][0]) + ', batch size - ' + str(res_2[8][1]) + ', dense layer size - ' + \
    str(res_2[8][2]) + ', amount of maps - ' + str(res_2[8][3]) + ', kernel 1 size - ' + str(res_2[8][4]) + \
    ', kernel 2 size - ' + str(res_2[8][5]) + ', kernel 3 size - ' + str(res_2[8][6]) + '\n')
print('Decaying e-greedy: amount of epochs - ' + str(res_3[8][0]) + ', batch size - ' + str(res_3[8][1]) + \
    ', dense layer size - ' + str(res_3[8][2]) + ', amount of maps - ' + str(res_3[8][3]) + ', kernel 1 size - ' + \
    str(res_3[8][4]) + ', kernel 2 size - ' + str(res_3[8][5]) + ', kernel 3 size - ' + str(res_3[8][6]))

In [ ]:
plt.plot(range(len(res_1[1])), res_1[1], linewidth = 8)
plt.plot(range(len(res_2[1])), res_2[1], linewidth = 4)
plt.plot(range(len(res_3[1])), res_3[1], linewidth = 2)
plt.grid()
plt.xlabel('Amount of episodes')
plt.ylabel('Amount of epochs')
plt.legend(['Greedy', 'e-greedy', 'Decaying e-greedy'])
plt.show()

In [ ]:
plt.plot(range(len(res_1[2])), res_1[2], linewidth = 8)
plt.plot(range(len(res_2[2])), res_2[2], linewidth = 4)
plt.plot(range(len(res_3[2])), res_3[2], linewidth = 2)
plt.grid()
plt.xlabel('Amount of episodes')
plt.ylabel('Batch size')
plt.legend(['Greedy', 'e-greedy', 'Decaying e-greedy'])
plt.show()

In [ ]:
plt.plot(range(len(res_1[3])), res_1[3], linewidth = 8)
plt.plot(range(len(res_2[3])), res_2[3], linewidth = 4)
plt.plot(range(len(res_3[3])), res_3[3], linewidth = 2)
plt.grid()
plt.xlabel('Amount of episodes')
plt.ylabel('Dense layer size')
plt.legend(['Greedy', 'e-greedy', 'Decaying e-greedy'])
plt.show()

In [ ]:
plt.plot(range(len(res_1[4])), res_1[4], linewidth = 8)
plt.plot(range(len(res_2[4])), res_2[4], linewidth = 4)
plt.plot(range(len(res_3[4])), res_3[4], linewidth = 2)
plt.grid()
plt.xlabel('Amount of episodes')
plt.ylabel('Amount of maps')
plt.legend(['Greedy', 'e-greedy', 'Decaying e-greedy'])
plt.show()

In [ ]:
plt.plot(range(len(res_1[5])), res_1[5], linewidth = 8)
plt.plot(range(len(res_2[5])), res_2[5], linewidth = 4)
plt.plot(range(len(res_3[5])), res_3[5], linewidth = 2)
plt.grid()
plt.xlabel('Amount of episodes')
plt.ylabel('Kernel 1 size')
plt.legend(['Greedy', 'e-greedy', 'Decaying e-greedy'])
plt.show()

In [ ]:
plt.plot(range(len(res_1[6])), res_1[6], linewidth = 8)
plt.plot(range(len(res_2[6])), res_2[6], linewidth = 4)
plt.plot(range(len(res_3[6])), res_3[6], linewidth = 2)
plt.grid()
plt.xlabel('Amount of episodes')
plt.ylabel('Kernel 2 size')
plt.legend(['Greedy', 'e-greedy', 'Decaying e-greedy'])
plt.show()

In [ ]:
plt.plot(range(len(res_1[7])), res_1[7], linewidth = 8)
plt.plot(range(len(res_2[7])), res_2[7], linewidth = 4)
plt.plot(range(len(res_3[7])), res_3[7], linewidth = 2)
plt.grid()
plt.xlabel('Amount of episodes')
plt.ylabel('Kernel 3 size')
plt.legend(['Greedy', 'e-greedy', 'Decaying e-greedy'])
plt.show()

## Alexander's algorithm

In [ ]:
import itertools
from collections import Counter

In [ ]:
def initialize_q_table(state_space, action_space, range_=(-1, 1), zeros=False):
    func = np.zeros if zeros else np.random.random
    qtable = (not zeros) * range_[0] + (range_[1] - range_[0]) * func((state_space, action_space))
    return qtable


def epsilon_greedy_policy(qtable, state, epsilon):
    # Randomly generate a number between 0 and 1
    random_num = np.random.random()
    # if random_num > greater than epsilon --> exploitation
    if random_num > epsilon:
        # Take the action with the highest value given a state
        # np.argmax can be useful here
        action = np.argmax(qtable[state])
    # else --> exploration
    else:
        action = np.random.choice(np.arange(len(qtable[0])))  # Take a random action
    return action


def rec_state_path(x, i, depth, path_):
    if i > 1000:
        i = 1000
    if i in path_:
        return
    path_.append(i)
    if depth == 0:
        return x[i]
    return rec_state_path(x, x[i], depth - 1, path_)


def rec_state(x, i, depth):
    if i > 1000:
        i = 1000
    if depth == 0:
        return x[i]
    return rec_state(x, x[i], depth - 1)


def get_best_params(qtable, env, calc_acc=True):
    x = [np.argmax(qtable[i]) for i in range(len(qtable))]
    y = [
        [rec_state(x, i, 2) for i in range(len(qtable))],
        [rec_state(x, i, 3) for i in range(len(qtable))]
    ]

    y1 = np.zeros(len(qtable[0]))
    for i in range(len(qtable[0])):
        path = []
        rec_state_path(x, i, len(qtable[0]), path)
        for p in path:
            y1[p] += 1
    states = [Counter(i).most_common()[0][0] for i in y] + [np.argmax(y1)]
    if calc_acc:
        accs = [env.step(i)[1] for i in states]
        return [[{k: v for k, v in zip(env.params_names, env.comb[state])}, acc] for state, acc in zip(states, accs)]
    else:
        return [{k: v for k, v in zip(env.params_names, env.comb[state])} for state in states]

In [ ]:
class ObjModel:
    def __init__(self, base_model, params, data):
        self.data = data
        self.params = params
        self.params_names = self.params.keys()
        self.comb = self.gen_comb()
        self.observation_space_n = len(self.comb)
        self.action_space_n = len(self.comb)
        self.base_model = base_model
        self.model = None
        self.reset()

    def reset(self):
        state = self.goto_state(0)
        return state

    def goto_state(self, state):
        prms = {k: v for k, v in zip(self.params_names, self.comb[state])}
        prms = np.array(list(prms.items()))[:, 1]
        prms = prms.astype(np.int)
        self.model = self.base_model((prms[2], prms[2]), (prms[3], prms[3]), (prms[4], prms[4]), prms[5], prms[6])
        return state

    def train(self, state):
        prms = {k: v for k, v in zip(self.params_names, self.comb[state])}
        prms = np.array(list(prms.items()))[:, 1]
        prms = prms.astype(np.int)
        tf.random.set_seed(7)
        self.model.fit(self.data[0], self.data[1], epochs = prms[0], batch_size = prms[1], validation_data = (tst_d, tst_l), 
            verbose = False)

    def reward(self):
        return self.model.evaluate(self.data[2], self.data[3], verbose = False)[1]

    def gen_comb(self):
        comb = list(itertools.product(*[self.params[i] for i in self.params]))
        return comb

    def step(self, action):
        state = self.goto_state(action)
        self.train(state)
        r = self.reward()
        return state, r


def train(epsilon, decay_rate, env, max_steps, qtable, gamma, learning_rate):
    episode_rewards = [[], []]
    reward_old = 0
    reward_best = 0
    # Reset the environment
    state = env.reset()

    # repeat
    for step in range(max_steps):
        # Choose the action At using epsilon greedy policy
        action = epsilon_greedy_policy(qtable, state, epsilon * np.exp(-decay_rate * step))

        # Take action At and observe Rt+1 and St+1
        # Take the action (a) and observe the outcome state(s') and reward (r)
        new_state, reward_ = env.step(action)
        reward = reward_ if reward_ >= reward_old else (reward_ - reward_old) * 10
        if reward_ > reward_best:
            if step > 0:
                reward *= 2
            reward_best = reward_
        reward_old = reward_

        action_ = epsilon_greedy_policy(qtable, new_state, epsilon)

        # Update Q(s,a)
        qtable[state][action] += learning_rate * (
                reward + gamma * qtable[new_state][action_] - qtable[state][action])

        # Our state is the new state
        state = new_state
        episode_rewards[0].append(reward_)
        episode_rewards[1].append(reward)
    return qtable, episode_rewards

In [ ]:
# Environment parameters
params = {
            'epochs': np.arange(5, 36, 10), 
            'batch_size': np.arange(5, 36, 10),
            'kernel_1': [3, 5, 7],
            'kernel_2': [3, 5, 7],
            'kernel_3': [3, 5, 7],
            'dense_layer': np.arange(70, 131, 30),
            'maps': np.arange(8, 25, 8)
        }
data = (trn_d, trn_l, tst_d, tst_l)
env = ObjModel(build_model, params, data)     # environment
# Training parameters
learning_rate = 1          # Learning rate
max_steps = 20            # Max steps per episode
gamma = 0.9                 # Discounting rate

# Exploration parameters
epsilon = 1.0
decay_rate = 0.005            # Exponential decay rate for exploration prob

In [ ]:
state_space = env.observation_space_n # TODO
action_space = env.action_space_n # TODO
Qtable_init_s = initialize_q_table(state_space, action_space)

Qtable_s, episod_rewards_s = train(epsilon, decay_rate, env, max_steps, Qtable_init_s, gamma, learning_rate)

params = get_best_params(Qtable_s, env, calc_acc=True)

In [ ]:
accs = [i[1] for i in params]
print(accs, 'best acc:', np.max(accs))

In [ ]:
print('best params:', params[np.argmax(accs)][0])

In [ ]:
#plt.plot(episod_rewards)
#plt.plot(np.cumsum(episod_rewards_s[1]))
plt.plot(episod_rewards_s[0])
#plt.plot(episod_rewards_s[1])
plt.grid()
plt.xlabel('Learning steps')
plt.ylabel('Accuracy')
plt.show()

In [ ]:
#plt.plot(episod_rewards)
plt.plot(np.cumsum(episod_rewards_s[1]))
#plt.plot(episod_rewards_s[0])
#plt.plot(episod_rewards_s[1])
plt.grid()
plt.xlabel('Learning steps')
plt.ylabel('Cumulative reward')
plt.show()